In [2]:
import pyspark

Like before, the first step is to initialize a spark context:

In [3]:
sc = pyspark.SparkContext()

Now we can initialize a SQL Context passing the <code>sc</code> object to the <code>SQLContext</code> function. The convention when initializing a <code>SQLContext</code> is to pass it to a variable called <code>spark</code>, though you will also see many examples where this variable is called <code>sql</code> or <code>SQLContext</code>.

In [4]:
from pyspark.sql import SQLContext

spark = SQLContext(sc)

The Spark SQL API, unlike the RDD API, has a method to read CSV files directly. Let's use it to load our museum data from the RDD API notebook. An alterntive syntax for the command below is <code>spark.read.format("csv").load('MetObjects.csv')</code>. Reading this you might be asking yourself if the SparkSQL API has methods to read other formats directly... and the answer is yes, it does. We will mostly focus on CSV today, but you can find a complete list of supported data sources here: https://spark.apache.org/docs/latest/sql-data-sources.html

In [13]:
museum_data = spark.read.options(header='true').csv('../../data/MetObjects.csv.gz')

The command above loaded our CSV into a data structure called a DataFrame. If you are familiar with R or Pandas, you will be right at home with the SparkSQL API! 

The <code>head</code> method shows you the first row of a DataFrame. Notice the word "Row" on the output: a DataFrame is an RDD where the elements are objects of the <code>Row</code> class!

In [7]:
museum_data.head()

Row(Object Number='1979.486.1', Is Highlight='False', Is Timeline Work='False', Is Public Domain='False', Object ID='1', Gallery Number=None, Department='The American Wing', AccessionYear='1979', Object Name='Coin', Title='One-dollar Liberty Head Coin', Culture=None, Period=None, Dynasty=None, Reign=None, Portfolio=None, Constiuent ID='16429', Artist Role='Maker', Artist Prefix=None, Artist Display Name='James Barton Longacre', Artist Display Bio='American, Delaware County, Pennsylvania 1794–1869 Philadelphia, Pennsylvania', Artist Suffix=None, Artist Alpha Sort='Longacre, James Barton', Artist Nationality='American', Artist Begin Date='1794      ', Artist End Date='1869      ', Artist Gender=None, Artist ULAN URL='http://vocab.getty.edu/page/ulan/500011409', Artist Wikidata URL=None, Object Date='1853', Object Begin Date='1853', Object End Date='1853', Medium='Gold', Dimensions='Dimensions unavailable', Credit Line='Gift of Heinz L. Stoppelmann, 1979', Geography Type=None, City=None, 

Another way of peeking into a DataFrame is the <code>show</code> method. This prints your DataFrame in a way analogous to how you'd get an output from a Relational Database Management Software (RDBMS) on the command line.

In [8]:
museum_data.show(5)

+-------------+------------+----------------+----------------+---------+--------------+-----------------+-------------+-----------+--------------------+-------+------+-------+-----+---------+-------------+-----------+-------------+--------------------+--------------------+-------------+--------------------+------------------+-----------------+---------------+-------------+--------------------+-------------------+-----------+-----------------+---------------+------+--------------------+--------------------+--------------+----+-----+------+-------+------+---------+------+-----+----------+-----+--------------+-----------------------+--------------------+-------------------+-------------+--------------------+----+------------+-----------------+
|Object Number|Is Highlight|Is Timeline Work|Is Public Domain|Object ID|Gallery Number|       Department|AccessionYear|Object Name|               Title|Culture|Period|Dynasty|Reign|Portfolio|Constiuent ID|Artist Role|Artist Prefix| Artist Display Na

A third option is using the method <code>toPandas</code>. As the name says, this will bring your DataFrame back to the Driver and convert it into a Pandas DataFrame. Notice the <code>limit</code> method called just before <code>toPandas</code>: since this is bringing data back from the Cluster to the Driver, you need to make sure you are not bringing too much back!

In [9]:
import pandas as pd

museum_data.limit(5).toPandas()

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,...,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,1979.486.1,False,False,False,1,None,The American Wing,1979,Coin,One-dollar Liberty Head Coin,...,None,Metal,None,http://www.metmuseum.org/art/collection/search/1,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None
1,1980.264.5,False,False,False,2,None,The American Wing,1980,Coin,Ten-dollar Liberty Head Coin,...,None,Metal,None,http://www.metmuseum.org/art/collection/search/2,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None
2,67.265.9,False,False,False,3,None,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,...,None,Metal,None,http://www.metmuseum.org/art/collection/search/3,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None
3,67.265.10,False,False,False,4,None,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,...,None,Metal,None,http://www.metmuseum.org/art/collection/search/4,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None
4,67.265.11,False,False,False,5,None,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,...,None,Metal,None,http://www.metmuseum.org/art/collection/search/5,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None


Now let's look into how to maipulate data with the SparkSQL API. We'll start with the DataFrames API component. It looks just like SQL, but instead of writing statements, you will call methods just like we did with the RDD API. For example, you can select a single column out of a DataFrame where a condition based on another column is satisfied, then count the number of rows returned:

In [22]:
museum_data.select("Title").where(museum_data.AccessionYear > 2018).count()

3437

Also like the RDD API, you can use the <code>take</code> method to bring a few rows back to the Driver and print the results:

In [44]:
museum_data.select("Title","Repository", "AccessionYear").where(museum_data.AccessionYear > 2018).take(5)

[Row(Title='Jubal and Miriam', Repository='Metropolitan Museum of Art, New York, NY', AccessionYear='2019'),
 Row(Title='Towel or drying rack', Repository='Metropolitan Museum of Art, New York, NY', AccessionYear='2020'),
 Row(Title='Vase', Repository='Metropolitan Museum of Art, New York, NY', AccessionYear='2019'),
 Row(Title='Venetian Summer', Repository='Metropolitan Museum of Art, New York, NY', AccessionYear='2019'),
 Row(Title='Covered box', Repository='Metropolitan Museum of Art, New York, NY', AccessionYear='2019')]

Similarly to how you'd do it using SQL, you can use the <code>distinct</code> method to return unique instances out of one or more columns:

In [20]:
museum_data.select("Artist Display Name").distinct().count()

63135

In [21]:
museum_data.select("Artist Display Name").distinct().take(5)

[Row(Artist Display Name='Marks Adjustable Folding Chair Company'),
 Row(Artist Display Name='Matsu-Zukaya Company|Frank Lloyd Wright'),
 Row(Artist Display Name='William F. Ladd'),
 Row(Artist Display Name='Edward Lycett|Faience Manufacturing Company'),
 Row(Artist Display Name='Boston & Sandwich Glass Company|Henry N. Hooper and Company')]

The <code>groupBy</code> method works like the "GROUP BY" SQL clause: use it to apply aggregations to your data based on one or more columns:

In [23]:
museum_data.groupBy("Artist Display Name").count().take(5)

[Row(Artist Display Name='Marks Adjustable Folding Chair Company', count=1),
 Row(Artist Display Name='Matsu-Zukaya Company|Frank Lloyd Wright', count=1),
 Row(Artist Display Name='William F. Ladd', count=6),
 Row(Artist Display Name='Edward Lycett|Faience Manufacturing Company', count=1),
 Row(Artist Display Name='Boston & Sandwich Glass Company|Henry N. Hooper and Company', count=1)]

In [24]:
museum_data.groupBy("Artist Display Name","AccessionYear").count().take(10)

[Row(Artist Display Name='Andrew Ellicott Warner', AccessionYear='1987', count=10),
 Row(Artist Display Name='John and Joseph W. Meeks', AccessionYear='1969', count=1),
 Row(Artist Display Name='Julius Dessoir', AccessionYear='1969', count=1),
 Row(Artist Display Name='Peleg Armstrong and Erastus Wentworth', AccessionYear='1918', count=1),
 Row(Artist Display Name='George Fielding', AccessionYear='1923', count=1),
 Row(Artist Display Name='W. H. T.', AccessionYear='1933', count=1),
 Row(Artist Display Name='William J. Elsworth', AccessionYear='1939', count=1),
 Row(Artist Display Name='Jeronimus Alstyne', AccessionYear='1933', count=2),
 Row(Artist Display Name='John White Alexander', AccessionYear='1891', count=1),
 Row(Artist Display Name='William M. S. Doyle', AccessionYear='1937', count=1)]

The <code>OrderBy</code> method works like the "ORDER BY" SQL statement: notice that its position in the chain of methods we call is analogous to where you'd place an ORDER BY statement in a SQL query!

In [63]:
museum_data.groupBy("Artist Display Name","AccessionYear").count().orderBy("Artist Display Name","AccessionYear").take(5)

[Row(Artist Display Name=None, AccessionYear=None, count=28163),
 Row(Artist Display Name=None, AccessionYear=' 1973"', count=1),
 Row(Artist Display Name=None, AccessionYear=' 1979"', count=1),
 Row(Artist Display Name=None, AccessionYear=' 1984"', count=1),
 Row(Artist Display Name=None, AccessionYear=' 1985"', count=1)]

In [74]:
museum_data.dropna(subset='Artist Display Name').groupBy("Artist Display Name","AccessionYear").count().orderBy("Artist Display Name","AccessionYear").take(5)

[Row(Artist Display Name=' ""The great Suspension Bridge"""', AccessionYear='1994', count=1),
 Row(Artist Display Name=' 7-9', AccessionYear='1924', count=1),
 Row(Artist Display Name=' candelabra', AccessionYear='1941', count=3),
 Row(Artist Display Name=' chapter viii', AccessionYear='1963', count=1),
 Row(Artist Display Name=' à source des Sciences"', AccessionYear='1917', count=2)]

Now, you must be wondering why we chose this Museum dataset for the workshop. The line above is why: this dataset is incredibly broken! Notice on the block above how none of the outputs look like an actual Artist Display Name. The next line gives us an idea of what is going on here - there are too many quotes in some of the coulmns! Look at the "Title" column to see what we mean.

In [9]:
museum_data.where(museum_data['Artist Display Name'] == ' candelabra').take(1)

[Row(Object Number='41.71.1.12(2)', Is Highlight='False', Is Timeline Work='False', Is Public Domain='True', Object ID='362011', Gallery Number=None, Department='Drawings and Prints', AccessionYear='1941', Object Name='Print', Title='"Large vase found at the Pantanello, Hadrian\'s Villa, Tivoli, in 1770 (The ""Warwick Vase', Culture='"" from Vasi', Period=' candelabri', Dynasty=' cippi', Reign=' sarcofagi', Portfolio=' tripodi', Constiuent ID=' lucerne', Artist Role=' ed ornamenti antichi disegnati ed incisi dal Cav. Gio. Batt. Piranesi', Artist Prefix=' Vol. I (Vases', Artist Display Name=' candelabra', Artist Display Bio=' grave stones', Artist Suffix=' sarcophagi', Artist Alpha Sort=' tripods', Artist Nationality=' lamps', Artist Begin Date=' and ornaments designed and etched by Cavalieri Giovanni Battista Piranesi)"', Artist End Date=None, Artist Gender=None, Artist ULAN URL=None, Artist Wikidata URL=None, Object Date='Vasi, candelabri, cippi, sarcofagi, tripodi, lucerne, ed orname

We will try to use Spark to fix this problem later. For now, let's demonstrate a couple more things you can do with the DataFrames API while also demonstrating other ways in which this dataset is broken... here we use the <code>like</code> method to retrieve instances where the Artist's Nationality is an URL. Notice the SQL syntax in the argument of <code>like</code>: the wildcard character to tell Spark we are looking for the pattern "http followed by anything" is "%".

In [15]:
museum_data.select("Artist Nationality").distinct().where(museum_data['Artist Nationality'].like('http%')).take(10)

[Row(Artist Nationality='https://www.wikidata.org/wiki/Q41176|https://www.wikidata.org/wiki/Q8441|https://www.wikidata.org/wiki/Q81054|https://www.wikidata.org/wiki/Q11446'),
 Row(Artist Nationality='https://www.wikidata.org/wiki/Q3957|https://www.wikidata.org/wiki/Q8502|https://www.wikidata.org/wiki/Q191163'),
 Row(Artist Nationality='https://www.wikidata.org/wiki/Q571|https://www.wikidata.org/wiki/Q676555|https://www.wikidata.org/wiki/Q642420'),
 Row(Artist Nationality='https://www.wikidata.org/wiki/Q289|https://www.wikidata.org/wiki/Q405|https://www.wikidata.org/wiki/Q333'),
 Row(Artist Nationality='https://www.wikidata.org/wiki/Q5113'),
 Row(Artist Nationality='https://www.wikidata.org/wiki/Q5113|https://www.wikidata.org/wiki/Q11946202|https://www.wikidata.org/wiki/Q80066'),
 Row(Artist Nationality='https://www.wikidata.org/wiki/Q7559|https://www.wikidata.org/wiki/Q1264081'),
 Row(Artist Nationality='https://www.wikidata.org/wiki/Q5937779|https://www.wikidata.org/wiki/Q43393537|htt

The <code>where</code> method returns rows where whatever is passed into it as an argument is true. To get an equivalent to SQL's "NOT LIKE" we negate the output of <code>like</code> with a "~" (tilda). Notice how there are outputs that don't make sense here too!

In [15]:
museum_data.select("Artist Nationality").distinct().where(~museum_data['Artist Nationality'].like('http%')).take(10)

[Row(Artist Nationality='Edmonds, Francis William'),
 Row(Artist Nationality='Deities|Buddhism'),
 Row(Artist Nationality='Spanish|Italian|French'),
 Row(Artist Nationality='American, born Ukraine'),
 Row(Artist Nationality='American|Japanese'),
 Row(Artist Nationality='Brady, Mathew B.'),
 Row(Artist Nationality='British|British, born Italy'),
 Row(Artist Nationality='French|Belgian|Austrian|German|British, active France'),
 Row(Artist Nationality='German|German|Swiss|Roman|German|German'),
 Row(Artist Nationality='Puvis de Chavannes, Pierre')]

The next line will give us an idea of what is going on here: look at the very first column, "Object Number". Its value is 'white cedar"'. That lone quote at the end tells us this is probably the end of a larger quoted line and it ended up here because the line contains a new-line character (\n)! 

In [8]:
museum_data.where(museum_data['Artist Nationality'].like('http%')).take(1)

[Row(Object Number='white cedar"', Is Highlight='37 x 30 1/4 x 19 1/2 in. (94 x 76.8 x 49.5 cm)', Is Timeline Work='Gift of Mrs. Russell Sage, 1909', Is Public Domain='Possibly made in|Possibly made in', Object ID='Guilford|Saybrook', Gallery Number=None, Department=None, AccessionYear='United States|United States', Object Name=None, Title=None, Culture=None, Period=None, Dynasty=None, Reign=None, Portfolio='Furniture', Constiuent ID=None, Artist Role='http://www.metmuseum.org/art/collection/search/2029', Artist Prefix=None, Artist Display Name=None, Artist Display Bio='Metropolitan Museum of Art, New York, NY', Artist Suffix='Birds|Flowers', Artist Alpha Sort='http://vocab.getty.edu/page/aat/300266506|http://vocab.getty.edu/page/aat/300132399', Artist Nationality='https://www.wikidata.org/wiki/Q5113|https://www.wikidata.org/wiki/Q506', Artist Begin Date=None, Artist End Date=None, Artist Gender=None, Artist ULAN URL=None, Artist Wikidata URL=None, Object Date=None, Object Begin Date=N

We will look into how to fix those problems shortly. Before we do that though, let's look at the reason why this API is called SparkSQL in the first place. Using the <code>registerTempTable</code> method you can turn your DataFrame into a full-fledged table and use actual SQL to query it!

In [5]:
museum_data.registerTempTable("Museum")

In [6]:
spark.sql("SELECT Title,Repository FROM Museum WHERE AccessionYear > 2018").take(5)

[Row(Title='Jubal and Miriam', Repository='Metropolitan Museum of Art, New York, NY'),
 Row(Title='Towel or drying rack', Repository='Metropolitan Museum of Art, New York, NY'),
 Row(Title='Vase', Repository='Metropolitan Museum of Art, New York, NY'),
 Row(Title='Venetian Summer', Repository='Metropolitan Museum of Art, New York, NY'),
 Row(Title='Covered box', Repository='Metropolitan Museum of Art, New York, NY')]

In [21]:
spark.sql("SELECT COUNT(Title) FROM Museum WHERE AccessionYear > 2018").show()

+------------+
|count(Title)|
+------------+
|        2722|
+------------+



In [12]:
spark.sql("SELECT DISTINCT * FROM Museum WHERE `Artist Display Name` == ' candelabra' LIMIT 1").show()

+-------------+------------+----------------+----------------+---------+--------------+-------------------+-------------+-----------+--------------------+------------+-----------+-------+----------+---------+-------------+--------------------+--------------+-------------------+------------------+-------------+-----------------+------------------+--------------------+---------------+-------------+---------------+-------------------+--------------------+-----------------+---------------+------+--------------------+--------------------+--------------+--------------------+-------+----------+----------+------+--------------------+--------------------+-------+----------+-----+--------------+-----------------------+-------------+-------------------+-------------+----------+----+------------+-----------------+
|Object Number|Is Highlight|Is Timeline Work|Is Public Domain|Object ID|Gallery Number|         Department|AccessionYear|Object Name|               Title|     Culture|     Period|Dynasty

In [15]:
spark.sql("SELECT DISTINCT * FROM Museum WHERE `Artist Display Name` LIKE 'http%' LIMIT 1").show()

+--------------------+--------------------+----------------+----------------+---------+--------------+--------------------+-------------+-----------+-----+--------------------+--------------------+-------+--------------------+---------+-------------+-----------+-------------+--------------------+------------------+-------------+-----------------+------------------+-----------------+--------------------+-------------+---------------+-------------------+-----------+-----------------+---------------+------+----------+-----------+--------------+----+-----+------+-------+------+---------+------+-----+----------+-----+--------------+-----------------------+-------------+-------------------+-------------+----------+----+------------+-----------------+
|       Object Number|        Is Highlight|Is Timeline Work|Is Public Domain|Object ID|Gallery Number|          Department|AccessionYear|Object Name|Title|             Culture|              Period|Dynasty|               Reign|Portfolio|Constiuen

You can add multiple tables to your active Spark SQLContext and work on them as you would on a Relational Database - Join tables, Union tables... To see what tables are current registered in your SQLContext, use the <code>tableNames</code> method:

In [7]:
spark.tableNames()

['museum']

We had seen how to register tables with <code>registerTempTable</code>. To remove tables from the session, use <code>dropTempTable</code>:

In [14]:
spark.dropTempTable('museum')

Whether your DataFrame is registered as a Table in the session, you can see its column names by checking the <code>columns</code> attribute:

In [16]:
museum_data.columns

['Object Number',
 'Is Highlight',
 'Is Timeline Work',
 'Is Public Domain',
 'Object ID',
 'Gallery Number',
 'Department',
 'AccessionYear',
 'Object Name',
 'Title',
 'Culture',
 'Period',
 'Dynasty',
 'Reign',
 'Portfolio',
 'Constiuent ID',
 'Artist Role',
 'Artist Prefix',
 'Artist Display Name',
 'Artist Display Bio',
 'Artist Suffix',
 'Artist Alpha Sort',
 'Artist Nationality',
 'Artist Begin Date',
 'Artist End Date',
 'Artist Gender',
 'Artist ULAN URL',
 'Artist Wikidata URL',
 'Object Date',
 'Object Begin Date',
 'Object End Date',
 'Medium',
 'Dimensions',
 'Credit Line',
 'Geography Type',
 'City',
 'State',
 'County',
 'Country',
 'Region',
 'Subregion',
 'Locale',
 'Locus',
 'Excavation',
 'River',
 'Classification',
 'Rights and Reproduction',
 'Link Resource',
 'Object Wikidata URL',
 'Metadata Date',
 'Repository',
 'Tags',
 'Tags AAT URL',
 'Tags Wikidata URL']

And you can remove columns from a DataFrame using the <code>drop</code> method. Adding columns to spark DataFrames is not as easy and we will see one way to do it later!

In [9]:
museum_data.drop('Object Number').drop('Is Highlight').limit(2).toPandas()

,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,...,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,False,False,1,None,The American Wing,1979,Coin,One-dollar Liberty Head Coin,None,None,...,None,Metal,None,http://www.metmuseum.org/art/collection/search/1,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None
1,False,False,2,None,The American Wing,1980,Coin,Ten-dollar Liberty Head Coin,None,None,...,None,Metal,None,http://www.metmuseum.org/art/collection/search/2,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None


If you are familiar with SQL, you must be feeling pretty at home right now with the SparkSQL API. However, you might be asking yourself... how you handle column types? In other words, how can I work in a Database where the tables have no schema?

It turns out Tables and DataFrames in Spark do have schemas! You can check the <code>schema</code> attriubte:

In [10]:
museum_data.schema

StructType(List(StructField(Object Number,StringType,true),StructField(Is Highlight,StringType,true),StructField(Is Timeline Work,StringType,true),StructField(Is Public Domain,StringType,true),StructField(Object ID,StringType,true),StructField(Gallery Number,StringType,true),StructField(Department,StringType,true),StructField(AccessionYear,StringType,true),StructField(Object Name,StringType,true),StructField(Title,StringType,true),StructField(Culture,StringType,true),StructField(Period,StringType,true),StructField(Dynasty,StringType,true),StructField(Reign,StringType,true),StructField(Portfolio,StringType,true),StructField(Constiuent ID,StringType,true),StructField(Artist Role,StringType,true),StructField(Artist Prefix,StringType,true),StructField(Artist Display Name,StringType,true),StructField(Artist Display Bio,StringType,true),StructField(Artist Suffix,StringType,true),StructField(Artist Alpha Sort,StringType,true),StructField(Artist Nationality,StringType,true),StructField(Artist 

In our case, wee did not specify what the schema of the DataFrame should be when we loaded the source CSV file, so Spark went ahead and read everything as Strings.

As it turns out, you can actually impose schemas on Spark DataFrames before you start populating them with data, in a way analogous to how you write a DDL statement to define a Table's schema in SQL.

Below you will see an example of what the schema of our museum data could look like. The following entities are worth of note: <code>StructType</code>, <code>StructField</code> and the different data Types. In SparkSQL, the <code>StructType</code> class defines what a Row will contain, and the <code>StructField</code> class defines what the columns in that row will look like.

The order of the StructFields in a StructType must match what is in the dataset you will import into your DataFrame.

In [9]:
from pyspark.sql.types import *

museum_schema = StructType([StructField('Object Number',StringType()), 
                     StructField('Is Highlight',     BooleanType()), 
                     StructField('Is Timeline Work', BooleanType()), 
                     StructField('Is Public Domain', BooleanType()),
                     StructField('Object ID',        IntegerType()),
                     StructField('Gallery Number',   IntegerType()),
                     StructField('Department',       StringType()),
                     StructField('AccessionYear',    IntegerType()),
                     StructField('ObjectName',       StringType()),
                     StructField('Ttile ',           StringType()),
                     StructField('Culture',          StringType()),
                     StructField('Period',           StringType()),
                     StructField('Dynasty',          StringType()),
                     StructField('Reign',            StringType()),
                     StructField('Portfolio',        StringType()),
                     StructField('Constituent ID',   IntegerType()),
                     StructField('Artist Role',      StringType()),
                     StructField('Artist Prefix',    StringType()),
                     StructField('Artist Display Name',StringType()),
                     StructField('Artist Display Bio',StringType()),
                     StructField('Artist Suffix',    StringType()),
                     StructField('Artist Alpha Sort',StringType()),
                     StructField('Artist Nationality',StringType()),
                     StructField('Artist Begin Date',IntegerType()),
                     StructField('Artist End Date',  IntegerType()),
                     StructField('Artist Gender',    StringType()),
                     StructField('Artist ULAN URL',  StringType()),
                     StructField('Artist Wikidata URL',StringType()),
                     StructField('Object Date',      DateType()),
                     StructField('Object Begin Date',IntegerType()),
                     StructField('Object End Date',  IntegerType()),
                     StructField('Object Display Name',StringType()),
                     StructField('Medium',           StringType()),
                     StructField('Dimensions',       StringType()),
                     StructField('Credit Line',      StringType()),
                     StructField('Geography Type',   StringType()),
                     StructField('City',             StringType()),
                     StructField('State',            StringType()),
                     StructField('County',           StringType()),
                     StructField('Country',          StringType()),
                     StructField('Region',           StringType()),
                     StructField('Subregion',        StringType()),
                     StructField('Locale',           StringType()),
                     StructField('Locus',            StringType()),
                     StructField('Excavation',       StringType()),
                     StructField('River',            StringType()),
                     StructField('Classification',   StringType()),
                     StructField('Rights and Reproduction',StringType()),
                     StructField('Link Resource',    StringType()),
                     StructField('Object Wikidata URL',StringType()),
                     StructField('Metadata Date',    DateType()),
                     StructField('Repository',       StringType()),
                     StructField('Tags',             StringType()),
                     StructField('Tags AAT URL',     StringType()),
                     StructField('Tags Wikidata URL',StringType())
                    ])

Now let's try importing our CSV file and impose the schema above schema on it. To do so, we'll add the "schema" parameter to our call to the <code>csv</code> method we had used before:

In [10]:
museum_data = spark.read.options(header='true').csv('../../data/MetObjects.csv.gz', schema=museum_schema)

museum_data.head()

Row(Object Number=None, Is Highlight=None, Is Timeline Work=None, Is Public Domain=None, Object ID=None, Gallery Number=None, Department=None, AccessionYear=None, ObjectName=None, Ttile =None, Culture=None, Period=None, Dynasty=None, Reign=None, Portfolio=None, Constituent ID=None, Artist Role=None, Artist Prefix=None, Artist Display Name=None, Artist Display Bio=None, Artist Suffix=None, Artist Alpha Sort=None, Artist Nationality=None, Artist Begin Date=None, Artist End Date=None, Artist Gender=None, Artist ULAN URL=None, Artist Wikidata URL=None, Object Date=None, Object Begin Date=None, Object End Date=None, Object Display Name=None, Medium=None, Dimensions=None, Credit Line=None, Geography Type=None, City=None, State=None, County=None, Country=None, Region=None, Subregion=None, Locale=None, Locus=None, Excavation=None, River=None, Classification=None, Rights and Reproduction=None, Link Resource=None, Object Wikidata URL=None, Metadata Date=None, Repository=None, Tags=None, Tags AAT

All of our data got loaded as <code>None</code>, which you can think of as the Python equivalent of a SQL <code>NULL</code>. We can confirm that it was indeed **all** of our data that gor loaded as NULLs running the line below. The <code>dropna</code> method, used without any arguments like we do below, filters out all rows containing NULLs from our DataFrame: 

In [41]:
museum_data.dropna().count()

0

So what happened here? Well, we know the dataset is broken and that there are likely lots of rows that won't possibly fit into the schema we specified. As it turns out, SparkSQL's default behaviour when reading data, is to fail when it comes across malformed inputs!

We can change that default behaviour by adding an option to drop malformed inputs and keep the good ones to our CSV reader:

In [12]:
museum_data = spark.read.options(header='true').option("mode","DROPMALFORMED").csv('../../data/MetObjects.csv.gz', schema=museum_schema)

museum_data.count()

5362

Looks like there is a whole lot of inputs that won't fit in our schema...

In general, enforcing schemas in SparkSQL is a good idea when you know you can trust the source to be **mostly** clean, i.e., that the number of columns and their types will mostly match the schema you intend to enforce. Having a well-defined schema in SparkSQL will enable you to use a number of typed built-in functions directly, without having to convert between types or wrangling data just so you can use a certain function. A notable example of when this comes in handy is whenever you are dealing with Dates or Timestamps.

Since we know our dataset is far from clean, we are better off not enforcing a schema and letting Spark decide instead. We add the option "inferschema" to tell Spark we'd like it to try to figure out from the data what the schema is. **Spoiler alert:** Spark does not infer schemas from messy data very reliably.

In [13]:
museum_data = spark.read.options(header='true').option("inferschema","true").option("mode","DROPMALFORMED").csv('../../data/MetObjects.csv.gz')
museum_data.count()

382209

Better, but still pretty far from the over 500k lines our CSV file actually has.

We had identified two problems with our data before:

1. Too many quotes in some lines.
2. Line breaks inside of quoted lines.

Let's try to solve these two problems now, and see if that means we can get more lines. First we will use the "multiline" option to let Spark know that there are some line breaks **in the fields** of our CSV file and they do not mean the next line is a different row of the dataset:

In [5]:
museum_data = spark.read.options(header='true').option("inferschema","true").option("multiline",True).option("mode","DROPMALFORMED").csv('../../data/MetObjects.csv.gz')
museum_data.count()

464607

Getting better! Let's see if this solved our problem with URLs showing up where Artists' nationalities should have been:

In [52]:
museum_data.select("Artist Nationality").distinct().where(museum_data['Artist Nationality'].like('http%')).take(10)

[]

Looks like at least there are no longer URLs in the Artist Nationality field. Let's see if those URLs are in the right field now:

In [16]:
museum_data.where(museum_data['Tags AAT URL']=='http://vocab.getty.edu/page/aat/300266506|http://vocab.getty.edu/page/aat/300386951|http://vocab.getty.edu/page/aat/300132410').take(1)

[Row(Object Number='29.100.762', Is Highlight='False', Is Timeline Work='False', Is Public Domain='False', Object ID='58673', Gallery Number=None, Department='Asian Art', AccessionYear='1929', Object Name='Inrō', Title=None, Culture='Japan', Period='Edo period (1615–1868)', Dynasty=None, Reign=None, Portfolio=None, Constiuent ID='16147', Artist Role='Artist', Artist Prefix=None, Artist Display Name='Louisine W. Havemeyer', Artist Display Bio=None, Artist Suffix=None, Artist Alpha Sort='Havemeyer, Louisine W.', Artist Nationality=None, Artist Begin Date='1855      ', Artist End Date='1929      ', Artist Gender='Female', Artist ULAN URL='http://vocab.getty.edu/page/ulan/500435594', Artist Wikidata URL='https://www.wikidata.org/wiki/Q539280', Object Date='19th century', Object Begin Date='1800', Object End Date='1899', Medium='Lacquer, gold, hirame, gold and coloured hiramakie, takamakie, nashiji; Interior: nashiji and fundame', Dimensions='3 5/16 x 1 15/16 x 1 in. (8.4 x 5 x 2.6 cm)', Cr

Bingo! Now let's see if we can solve the too-many-quotes problem. We do this by adding the "escape" option, indicating that any occurences of the " character inside a field, is to be read as a simple character and not a quote in the CSV-altering sense:

In [17]:
museum_data = spark.read.options(header='true').option("inferschema","true").option("escape", "\"").option("multiline",True).option("mode","DROPMALFORMED").csv('../../data/MetObjects.csv.gz')
museum_data.count()

474526

Got an extra 10k lines, not bad! Let's check what one of our previously broken lines look like now:

In [18]:
museum_data.where(museum_data['Artist Display Name'] == ' candelabra').take(1)

[]

In [41]:
museum_data.where(museum_data['Object ID'] == '362011').take(1)

[Row(﻿Object Number='41.71.1.12(2)', Is Highlight=False, Is Timeline Work=False, Is Public Domain=True, Object ID=362011, Gallery Number=None, Department='Drawings and Prints', AccessionYear='1941', Object Name='Print', Title='Large vase found at the Pantanello, Hadrian\'s Villa, Tivoli, in 1770 (The "Warwick Vase," from Vasi, candelabri, cippi, sarcofagi, tripodi, lucerne, ed ornamenti antichi disegnati ed incisi dal Cav. Gio. Batt. Piranesi, Vol. I (Vases, candelabra, grave stones, sarcophagi, tripods, lamps, and ornaments designed and etched by Cavalieri Giovanni Battista Piranesi)', Culture=None, Period=None, Dynasty=None, Reign=None, Portfolio='Vasi, candelabri, cippi, sarcofagi, tripodi, lucerne, ed ornamenti antichi disegnati ed incisi dal Cav. Gio. Batt. Piranesi, Vol. I', Constiuent ID=16225.0, Artist Role='Artist', Artist Prefix=None, Artist Display Name='Giovanni Battista Piranesi', Artist Display Bio='Italian, Mogliano Veneto 1720–1778 Rome', Artist Suffix=None, Artist Alph

The key takeaway here is that many data quality problems can be solved in Spark at load time. This is true of most other data formats supported by Spark too! So if you come across problems like the ones we've seen here where records are potentially malformed, a good first step is always to use one of the <code>read</code> methods and their options to solve them. For more on these methods and their options, see here: https://spark.apache.org/docs/latest/sql-data-sources.html

There are many other problems like these left in this dataset and you are welcome to try and find them. They should be possible to fix by adding options to the CSV reader like we did above.

If however you can't solve a problem with one of the <code>read</code> methods, or if you need to transform a field that got loaded successfully (but not correctly) or if you need to add a new column to your DataFrame based on existing columns, a powerful tool at your disposal is... the RDD API!

You can extract an RDD from a DataFrame by reading the <code>rdd</code> attribute like this:

In [27]:
museum_rdd = museum_data.rdd

A DataFrame, however, is an RDD where the elements are objects of the class Row as we've seen before:

In [7]:
museum_rdd.take(1)

[Row(﻿Object Number='1979.486.1', Is Highlight=False, Is Timeline Work=False, Is Public Domain=False, Object ID=1, Gallery Number=None, Department='The American Wing', AccessionYear='1979', Object Name='Coin', Title='One-dollar Liberty Head Coin', Culture=None, Period=None, Dynasty=None, Reign=None, Portfolio=None, Constiuent ID=16429.0, Artist Role='Maker', Artist Prefix=None, Artist Display Name='James Barton Longacre', Artist Display Bio='American, Delaware County, Pennsylvania 1794–1869 Philadelphia, Pennsylvania', Artist Suffix=None, Artist Alpha Sort='Longacre, James Barton', Artist Nationality='American', Artist Begin Date='1794      ', Artist End Date='1869      ', Artist Gender=None, Artist ULAN URL='http://vocab.getty.edu/page/ulan/500011409', Artist Wikidata URL=None, Object Date='1853', Object Begin Date=1853, Object End Date=1853, Medium='Gold', Dimensions='Dimensions unavailable', Credit Line='Gift of Heinz L. Stoppelmann, 1979', Geography Type=None, City=None, State=None

This isn't very useful outside of the SparkSQL API, so we convert all elements to good-old Python <code>lists</code> next: 

In [8]:
museum_rdd.map(list).take(1)

[['1979.486.1',
  False,
  False,
  False,
  1,
  None,
  'The American Wing',
  '1979',
  'Coin',
  'One-dollar Liberty Head Coin',
  None,
  None,
  None,
  None,
  None,
  16429.0,
  'Maker',
  None,
  'James Barton Longacre',
  'American, Delaware County, Pennsylvania 1794–1869 Philadelphia, Pennsylvania',
  None,
  'Longacre, James Barton',
  'American',
  '1794      ',
  '1869      ',
  None,
  'http://vocab.getty.edu/page/ulan/500011409',
  None,
  '1853',
  1853,
  1853,
  'Gold',
  'Dimensions unavailable',
  'Gift of Heinz L. Stoppelmann, 1979',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'Metal',
  None,
  'http://www.metmuseum.org/art/collection/search/1',
  None,
  None,
  'Metropolitan Museum of Art, New York, NY',
  None,
  None,
  '\r']]

Now, remember from before how we said that adding columns to DataFrames was not as straightforward as it sounds? There are actually many ways to do it, none being particularly easy or obvious to figure out. The one we recommend for the flexibility it gives the programmer is using the RDD API. On the next line, we will append a '1' to each element of our RDD if the Artist's Nationality is "Canadian", a '0' if the artist is not Canadian, and an 'NA' if the artist's nationality is NULL:

In [28]:
museum_rdd = museum_rdd.map(list).map(lambda row : row + ['NA' if not row[22] else '1' if "Canadian" in row[22] else '0'])

Now let's create a Python list containing the original column names from our DataFrame, plus a new column name: 'Is Canadian'.

In [33]:
column_names = museum_data.columns + ['Is Canadian']

Finally, we convert the RDD back to a DataFrame. Passing our list of column names as an argument tells Spark that this should be the header of the new DataFrame. The second argument tells Spark how much of the RDD it should look at in order to try to figure out the schema of the DataFrame. If your dataset is massive, even 1% (0.01) might be too much and take a long time to run!

In [35]:
museum_data = museum_rdd.toDF(column_names,sampleRatio=0.01)

In [36]:
museum_data.take(1)

[Row(﻿Object Number='1979.486.1', Is Highlight=False, Is Timeline Work=False, Is Public Domain=False, Object ID=1, Gallery Number=None, Department='The American Wing', AccessionYear='1979', Object Name='Coin', Title='One-dollar Liberty Head Coin', Culture=None, Period=None, Dynasty=None, Reign=None, Portfolio=None, Constiuent ID=16429.0, Artist Role='Maker', Artist Prefix=None, Artist Display Name='James Barton Longacre', Artist Display Bio='American, Delaware County, Pennsylvania 1794–1869 Philadelphia, Pennsylvania', Artist Suffix=None, Artist Alpha Sort='Longacre, James Barton', Artist Nationality='American', Artist Begin Date='1794      ', Artist End Date='1869      ', Artist Gender=None, Artist ULAN URL='http://vocab.getty.edu/page/ulan/500011409', Artist Wikidata URL=None, Object Date='1853', Object Begin Date=1853, Object End Date=1853, Medium='Gold', Dimensions='Dimensions unavailable', Credit Line='Gift of Heinz L. Stoppelmann, 1979', Geography Type=None, City=None, State=None

This concludes our introduction to the SparkSQL API. This API is extremely powerful and there is a lot more that you can do with it than we covered 